In [1]:
import nltk
from collections import Counter
import pandas as pd
import csv
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.tokenize import regexp_tokenize, wordpunct_tokenize,blankline_tokenize
from nltk import PorterStemmer, LancasterStemmer, SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.util import ngrams
import re
import string
import json
import re as regex
# This will score 0.556+ on the LB, you can further raise this score
# by using PCA and SVD as features.

# From https://github.com/rhiever/datacleaner
from datacleaner import autoclean
from sklearn.model_selection import GridSearchCV,train_test_split, cross_val_score, RandomizedSearchCV
from xgboost import XGBRegressor
from sklearn.metrics import r2_score

import numpy as np
import time







In [17]:
class TwitterData_Initialize():
    data = []
    processed_data = []
    wordlist = []

    featureList = []
    fea_vect=[]
    
    data_model = None
    data_labels = None
    is_testing = False
    
    def initialize(self, csv_file, is_testing_set=False, from_cached=None):
        if from_cached is not None:
            #self.data_model = pd.read_csv(from_cached)
            self.data_model = pd.read_json(from_cached)

            return

        self.is_testing = is_testing_set

        if not is_testing_set:
            #self.data = pd.read_csv(csv_file, header=0, names=["id", "emotion", "text"])
            self.data = pd.read_json(csv_file)

            #self.data = self.data[self.data["emotion"].isin(["positive", "negative", "neutral"])]
        

        self.processed_data = self.data
        self.wordlist = []
        self.data_model = None
        self.data_labels = None
        
    
    
    
    def do_process(self):
        start_time = time.time()
        def stem_and_join(row,stemmer=nltk.PorterStemmer()):
            row["spans"] = list(map(lambda str: stemmer.stem(str.lower()), row["spans"]))
            return row
    
        def tokenize_grams(row):
                
                # turn a doc into clean tokens
                def clean_doc(doc):
                    # split into tokens by white space
                    tokens = doc.split()
                    # remove punctuation from each token
                    table = str.maketrans('', '', string.punctuation)
                    tokens = [w.translate(table) for w in tokens]
                    # remove remaining tokens that are not alphabetic
                    tokens = [word for word in tokens if word.isalpha()]
                    # filter out stop words
                    #stop_words = set(stopwords.words('english'))
                    
                    #tokens = [w for w in tokens if not w in stop_words]
                    
                    
                    # filter out short tokens
                    tokens = [word.lower() for word in tokens if len(word) > 1]
                    return tokens
                




               
                # Function to apply lemmatization to a list of words
                def words_lemmatizer(words, encoding="utf8"):
                    wordnet_lemmatizer = WordNetLemmatizer()

                    lemma_words = []
                    wl = WordNetLemmatizer()
                    for word in words:
                        pos = find_pos(word)
                        lemma_words.append(wl.lemmatize(word, pos))
                    return lemma_words
                
                # Function to find part of speech tag for a word
                def find_pos(word):
                    # Part of Speech constants
                    # ADJ, ADJ_SAT, ADV, NOUN, VERB = 'a', 's', 'r', 'n', 'v'
   
                    pos = nltk.pos_tag(nltk.word_tokenize(word))[0][1]
                    # Adjective tags - 'JJ', 'JJR', 'JJS'
                    if pos.lower()[0] == 'j':
                        return 'a'
                    # Adverb tags - 'RB', 'RBR', 'RBS'
                    elif pos.lower()[0] == 'r':
                        return 'r'
                    # Verb tags - 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'
                    elif pos.lower()[0] == 'v':
                        return 'v'
                    # Noun tags - 'NN', 'NNS', 'NNP', 'NNPS'
                    else:
                        return 'n'
    
            
                #convert to string
                idx=row["spans"]
                #ch="".join(x for x in idx if x)
                ch=' '.join(idx)
               
    
                
                
                n_grams=clean_doc(ch)
                tokens=words_lemmatizer(n_grams)

                #print("nombre of tokens",len(n_grams))

                row["token_spans"] = tokens
    
                return row
        #self.processed_data = self.processed_data.apply(stem_and_join, axis=1)
        self.processed_data = self.processed_data.apply(tokenize_grams, axis=1)
        print("--- %s seconds ---" % (time.time() - start_time))

    

    def build_wordlist(self, min_occurrences=0, max_occurences=500, stopwords=nltk.corpus.stopwords.words("english"),
                       ):
       
        self.wordlist = []
        whitelist=["to","on","for","up","below","short","long"]
        
        #stopwords=[]
       
        words = Counter()
      
        for idx in self.processed_data.index:
            words.update(self.processed_data.loc[idx, "token_spans"])

        for idx, stop_word in enumerate(stopwords):
            if stop_word not in whitelist:
                del words[stop_word]
        
        
        print(words.most_common())        

        word_df = pd.DataFrame(data={"word": [k for k, v in words.most_common() if min_occurrences < v < max_occurences],
                                     "occurrences": [v for k, v in words.most_common() if min_occurrences < v < max_occurences]},
                               columns=["word", "occurrences"])

        word_df.to_csv("wordlist_copie.csv", index_label="idx")
        self.wordlist = [k for k, v in words.most_common() if min_occurrences < v < max_occurences]
        
        
        
    def build_data_model(self):
        label_column = []
        Id_column=["ID"]
        label_column = ["label"]

        columns = Id_column + label_column + list(
            map(lambda w: w ,self.wordlist))
        labels = []
        rows = []
        for idx in self.processed_data.index:
            current_row = []

            if True:
                # add label
                current_label = self.processed_data.loc[idx, "sentiment score"]
                current_id = self.processed_data.loc[idx, "id"]
                
                labels.append(current_id)
                labels.append(current_label)
                
                current_row.append(current_id)
                current_row.append(current_label)

            # add bag-of-words
            tokens = set(self.processed_data.loc[idx, "token_spans"])
            for _, word in enumerate(self.wordlist):
                current_row.append(1 if word in tokens else 0)

            rows.append(current_row)

        self.data_model = pd.DataFrame(rows, columns=columns)
        self.data_labels = pd.Series(labels)
        return self.data_model, self.data_labels


In [18]:
class TwitterData_Initialize_test(TwitterData_Initialize):

    
    
    def do_process(self):
        def stem_and_join(row,stemmer=nltk.PorterStemmer()):
            
            row["spans"] = list(map(lambda str: stemmer.stem(str.lower()), row["spans"]))
            return row
    
        def tokenize_grams(row):
                
                # Function to remove stop words
                def remove_stopwords(text, lang='english'):
                    whitelist = ["n't","not","below"]    

                    stop_words = set(stopwords.words('english'))
                    word_tokens = word_tokenize(text)
                    #filtered_sentence = [w for w in word_tokens if ((not w in stop_words) or (w in whitelist))]
                    filtered_sentence = []
                    for w in word_tokens:
                        if ((w not in stop_words) or (w in whitelist)):
                            filtered_sentence.append(w)
                
                    ch=" ".join(filtered_sentence)

                    return ch
#40

                def clean_doc(doc):
                    # split into tokens by white space
                    tokens = doc.split()
                    # remove punctuation from each token
                    table = str.maketrans('', '', string.punctuation)
                    tokens = [w.translate(table) for w in tokens]
                    # remove remaining tokens that are not alphabetic
                    tokens = [word for word in tokens if word.isalpha()]
                    # filter out stop words
                    #stop_words = set(stopwords.words('english'))
                    #tokens = [w for w in tokens if not w in stop_words]
                    
                    # filter out short tokens
                    tokens = [word.lower() for word in tokens if len(word) > 1]
                    return tokens


                

                # Function to apply lemmatization to a list of words
                def words_lemmatizer(words, encoding="utf8"):
                    wordnet_lemmatizer = WordNetLemmatizer()

                    lemma_words = []
                    wl = WordNetLemmatizer()
                    for word in words:
                        pos = find_pos(word)
                        lemma_words.append(wl.lemmatize(word, pos))
                    return lemma_words

                # Function to find part of speech tag for a word
                def find_pos(word):
                    # Part of Speech constants
                    # ADJ, ADJ_SAT, ADV, NOUN, VERB = 'a', 's', 'r', 'n', 'v'
   
                    pos = nltk.pos_tag(nltk.word_tokenize(word))[0][1]
                    # Adjective tags - 'JJ', 'JJR', 'JJS'
                    if pos.lower()[0] == 'j':
                        return 'a'
                    # Adverb tags - 'RB', 'RBR', 'RBS'
                    elif pos.lower()[0] == 'r':
                        return 'r'
                    # Verb tags - 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'
                    elif pos.lower()[0] == 'v':
                        return 'v'
                    # Noun tags - 'NN', 'NNS', 'NNP', 'NNPS'
                    else:
                        return 'n'
    
            
                token=[]
                idx=row["spans"]
                ch=' '.join(idx)


    
                #words_stemmer(ch, type="PorterStemmer", lang="english", encoding="utf8")
                #Convert to lower case
                n_grams=clean_doc(ch)
                tokens=words_lemmatizer(n_grams)

                #print("list ngrams--------------\n",n_grams)
                row["token_spans"] = tokens
    
                return row
        #self.processed_data = self.processed_data.apply(stem_and_join, axis=1)
        self.processed_data = self.processed_data.apply(tokenize_grams, axis=1)
        
    def build_wordlist(self, min_occurrences=0, max_occurences=500, stopwords=nltk.corpus.stopwords.words("english"),
                      ):
        
        whitelist=["to","on","for","up","below","short","long"]
        self.wordlist = []
        #stopwords=[]
        words = Counter()
      
        for idx in self.processed_data.index:
            words.update(self.processed_data.loc[idx, "token_spans"])

        for idx, stop_word in enumerate(stopwords):
            if stop_word not in whitelist:
                del words[stop_word]
        
        
        print(words.most_common())        

        word_df = pd.DataFrame(data={"word": [k for k, v in words.most_common() if min_occurrences < v < max_occurences],
                                     "occurrences": [v for k, v in words.most_common() if min_occurrences < v < max_occurences]},
                               columns=["word", "occurrences"])

        word_df.to_csv("wordlist_test.csv", index_label="idx")
        self.wordlist = [k for k, v in words.most_common() if min_occurrences < v < max_occurences]
        
        
    def build_data_model(self):
        
        label_id = ["ID"]

        columns = label_id + list(
            map(lambda w: w ,self.wordlist))
        labels = []
        rows = []
        for idx in self.processed_data.index:
            current_row = []
            if True:
                # add label
                current_id = self.processed_data.loc[idx, "id"]
                
                labels.append(current_id)
                
                current_row.append(current_id)

            

            # add bag-of-words
            tokens = set(self.processed_data.loc[idx, "token_spans"])
            for _, word in enumerate(self.wordlist):
                current_row.append(1 if word in tokens else 0)

            rows.append(current_row)

        self.data_model = pd.DataFrame(rows, columns=columns)
        return self.data_model


In [19]:
data = TwitterData_Initialize()
data.initialize("Microblog_Trainingdata.json")

len(data.processed_data)

1700

In [20]:
data_test = TwitterData_Initialize_test()
data_test.initialize("Microblog_Trialdata.json")

data_test.processed_data.head(5)
len(data_test.processed_data)

16

In [21]:
data.do_process()

--- 285.96952724456787 seconds ---


In [22]:
data.processed_data.head(5)

,cashtag,id,sentiment score,source,spans,token_spans
0,$FB,719659409228451840,0.366,twitter,[watching for bounce tomorrow],"[watch, for, bounce, tomorrow]"
1,$LUV,719904304207962112,0.638,twitter,[record number of passengers served in 2015],"[record, number, of, passenger, serve, in]"
2,$NFLX,5329774,-0.494,stocktwits,[out $NFLX -.35],"[out, nflx]"
3,$DIA,719891468173844480,0.460,twitter,"[Looking for a strong bounce, Lunchtime rally ...","[look, for, strong, bounce, lunchtime, rally, ..."
4,$PLUG,20091246,0.403,stocktwits,[Very intrigued with the technology and growth...,"[very, intrigue, with, the, technology, and, g..."


In [23]:
data_test.do_process()

In [24]:
data_test.processed_data.head(5)


,cashtag,id,sentiment score,source,spans,token_spans
0,$F,5540055,-0.454,stocktwits,[Putting on a little $F short],"[put, on, little, short]"
1,$AAPL,10752226,-0.464,stocktwits,[short some],"[short, some]"
2,$BAC,10920221,0.445,stocktwits,[buying opportunity],"[buying, opportunity]"
3,$SHOR,12971398,0.661,stocktwits,[Scaling Up on Long Position],"[scale, up, on, long, position]"
4,$JPM,16142438,-0.763,stocktwits,[its time to sell banks],"[it, time, to, sell, bank]"


In [25]:
data.build_wordlist()
data.processed_data.head(5)

[('to', 235), ('on', 133), ('for', 128), ('stock', 126), ('up', 109), ('short', 95), ('look', 83), ('buy', 76), ('long', 74), ('today', 68), ('sell', 66), ('high', 66), ('call', 59), ('go', 54), ('market', 49), ('good', 47), ('nice', 44), ('new', 44), ('still', 43), ('get', 43), ('day', 43), ('add', 42), ('week', 38), ('top', 38), ('bullish', 37), ('like', 33), ('put', 32), ('close', 31), ('big', 30), ('rally', 29), ('hold', 29), ('low', 29), ('trade', 28), ('take', 28), ('move', 27), ('break', 26), ('breakout', 26), ('strong', 25), ('tsla', 25), ('setup', 25), ('price', 25), ('buying', 24), ('see', 24), ('fb', 24), ('play', 24), ('aapl', 24), ('run', 23), ('support', 23), ('back', 23), ('start', 23), ('come', 22), ('time', 22), ('could', 22), ('downgrade', 22), ('weak', 21), ('lead', 21), ('bad', 20), ('one', 20), ('bull', 20), ('make', 20), ('growth', 19), ('overbought', 19), ('may', 19), ('share', 19), ('year', 19), ('next', 18), ('great', 18), ('sale', 18), ('well', 18), ('sector',

,cashtag,id,sentiment score,source,spans,token_spans
0,$FB,719659409228451840,0.366,twitter,[watching for bounce tomorrow],"[watch, for, bounce, tomorrow]"
1,$LUV,719904304207962112,0.638,twitter,[record number of passengers served in 2015],"[record, number, of, passenger, serve, in]"
2,$NFLX,5329774,-0.494,stocktwits,[out $NFLX -.35],"[out, nflx]"
3,$DIA,719891468173844480,0.460,twitter,"[Looking for a strong bounce, Lunchtime rally ...","[look, for, strong, bounce, lunchtime, rally, ..."
4,$PLUG,20091246,0.403,stocktwits,[Very intrigued with the technology and growth...,"[very, intrigue, with, the, technology, and, g..."


In [26]:
bow, labels = data.build_data_model()
bow.head(5)
len(bow.columns)

1844

In [27]:
data_test.processed_data.head()

,cashtag,id,sentiment score,source,spans,token_spans
0,$F,5540055,-0.454,stocktwits,[Putting on a little $F short],"[put, on, little, short]"
1,$AAPL,10752226,-0.464,stocktwits,[short some],"[short, some]"
2,$BAC,10920221,0.445,stocktwits,[buying opportunity],"[buying, opportunity]"
3,$SHOR,12971398,0.661,stocktwits,[Scaling Up on Long Position],"[scale, up, on, long, position]"
4,$JPM,16142438,-0.763,stocktwits,[its time to sell banks],"[it, time, to, sell, bank]"


In [28]:
data_test.build_wordlist()
len(data_test.wordlist)

[('on', 4), ('short', 4), ('put', 3), ('up', 3), ('long', 3), ('little', 2), ('buying', 2), ('position', 2), ('time', 2), ('to', 2), ('for', 2), ('opportunity', 1), ('scale', 1), ('sell', 1), ('bank', 1), ('enter', 1), ('picked', 1), ('accumulate', 1), ('far', 1), ('upside', 1), ('downside', 1), ('look', 1), ('strong', 1), ('bounce', 1), ('lunchtime', 1), ('rally', 1), ('come', 1), ('intrigue', 1), ('technology', 1), ('growth', 1), ('potential', 1), ('work', 1), ('big', 1), ('market', 1), ('loser', 1), ('goog', 1), ('googl', 1), ('would', 1), ('suck', 1), ('sbux', 1), ('dip', 1), ('below', 1), ('overbought', 1), ('dont', 1)]


44

In [29]:
bow_test= data_test.build_data_model()
bow_test.head(5)
len(bow_test.columns)

45

In [30]:
bow.head()

,ID,label,to,on,for,stock,up,short,look,buy,...,infect,malware,brings,upscale,concept,ownitdonttradeit,skin,gamepositive,overweight,callput
0,719659409228451840,0.366,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,719904304207962112,0.638,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,5329774,-0.494,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,719891468173844480,0.460,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,20091246,0.403,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
bow=bow.rename(columns={"label":"y"})
bow.head(5)


,ID,y,to,on,for,stock,up,short,look,buy,...,infect,malware,brings,upscale,concept,ownitdonttradeit,skin,gamepositive,overweight,callput
0,719659409228451840,0.366,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,719904304207962112,0.638,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,5329774,-0.494,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,719891468173844480,0.460,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,20091246,0.403,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
bow_test.head(5)


,ID,on,short,put,up,long,little,buying,position,time,...,loser,goog,googl,would,suck,sbux,dip,below,overbought,dont
0,5540055,1,1,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,10752226,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,10920221,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,12971398,1,0,0,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,16142438,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [33]:
# Concat the data frames

# This is for autocleaner to work properly

# Since we have categorical variables we will
# need our encoder to label them correctly
# so we must use our encoder on the full
# dataset to avoid having representation
# errors.
bow_clean = bow.append(bow_test)
bow_clean = autoclean(bow_clean)
#bow_clean = bow_clean.rename(columns = {'fit': 'fit_'})

train, test = bow_clean[0:len(bow)], bow_clean[len(bow):]

In [34]:
train.head(5)

,ID,aal,aapl,aapls,absolute,abyss,accept,acct,accumulate,accumulation,...,yoku,zero,zigzag,ziv,zmh,znga,zombie,zone,zuck,zuckerberg
0,719659409228451840,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,719904304207962112,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5329774,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,719891468173844480,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,20091246,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
test.head(5)

,ID,aal,aapl,aapls,absolute,abyss,accept,acct,accumulate,accumulation,...,yoku,zero,zigzag,ziv,zmh,znga,zombie,zone,zuck,zuckerberg
0,5540055,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10752226,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10920221,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,12971398,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,16142438,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
# Organize our data for training
X = train.drop({"y","ID"}, axis=1)
Y = train["y"]
#x_test = test.drop({"y","ID"}, axis=1)
#X, X_Val, Y, Y_Val = train_test_split(X, Y)
#print(type(Y_Val))

In [37]:
#print("X=",len(X),"Y=", len(X_Val),"X_val=", len(Y),"Y_val=", len(Y_Val))

In [38]:
#Y_Val.head(5)

In [39]:
"""
# A parameter grid for XGBoost
params = {'min_child_weight':[4,5], 'gamma':[i/10.0 for i in range(3,6)],  'subsample':[i/10.0 for i in range(6,11)],
'colsample_bytree':[i/10.0 for i in range(6,11)], 'max_depth': [2,3,4]}
# Initialize XGB and GridSearch
xgb = XGBRegressor(nthread=-1) 

grid = GridSearchCV(xgb, params)
grid.fit(X, Y)
"""

"\n# A parameter grid for XGBoost\nparams = {'min_child_weight':[4,5], 'gamma':[i/10.0 for i in range(3,6)],  'subsample':[i/10.0 for i in range(6,11)],\n'colsample_bytree':[i/10.0 for i in range(6,11)], 'max_depth': [2,3,4]}\n# Initialize XGB and GridSearch\nxgb = XGBRegressor(nthread=-1) \n\ngrid = GridSearchCV(xgb, params)\ngrid.fit(X, Y)\n"

In [40]:
import xgboost as xgb
from sklearn import model_selection, preprocessing
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
#---------------------------XGBregressor---------------------------------------------------

#Fitting XGB regressor with parameters obtained by Grid searchCV
"""
param_test1 = {
 'max_depth':range(3,10,2),
 'min_child_weight':range(1,6,2)
}
param_test2 = {
 'max_depth':[4,5,6],
 'min_child_weight':[4,5,6]
}
param_test3 = {
    'gamma':[i/10.0 for i in range(0,5)],
    'n_estimators' : [int(x) for x in np.linspace(start = 200, stop = 1400, num = 7)]
}

n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
model = GridSearchCV(estimator = XGBRegressor( learning_rate =0.1, max_depth=4,
 min_child_weight=4,  subsample=0.8, colsample_bytree=1.0, nthread=4), 
 param_grid = param_test3,scoring='neg_mean_squared_error',cv=5)
model.fit(X_train,y_train)

"""
#model = xgb.XGBRegressor(max_depth=4, min_child_weight= 4,gamma=0.3,subsample=0.8,colsample_bytree=1.0,n_estimators=1000)
model=xgb.XGBRegressor(learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 nthread=4,
 scale_pos_weight=1,
 seed=27)
model.fit(X_train,y_train)

print (model)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.7, gamma=0.3, learning_rate=0.1,
       max_delta_step=0, max_depth=4, min_child_weight=4, missing=None,
       n_estimators=100, n_jobs=1, nthread=None, objective='reg:linear',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=0.8)


In [41]:
# Print the r2 score
#print(r2_score(Y_Val, grid.best_estimator_.predict(X_Val))) 
x_test = test.drop({"y","ID"}, axis=1)
y_test=test["y"]

In [42]:
#Predict 
output = model.predict(x_test)
final_df = pd.DataFrame()
#final_df["ID"] = id_vals
#final_df["cashtag"]=row2

#final_df["spans"]=row
final_df["Prediction"] = output
#final_df.to_csv("Output_1.csv",sep=",")
final_df.head(20)

,Prediction
0,-0.454197
1,-0.254624
2,0.140863
3,0.527454
4,-0.233107
5,0.350240
6,0.244789
7,0.487672
8,0.429773
9,0.203284


In [43]:
# Save the file
#y_test = grid.best_estimator_.predict(x_test)
#results_df = pd.DataFrame(data={'y':y_test}) 
#ids = test_df["ID"]
#ids=bow_test.index
#joined = pd.DataFrame(ids).join(results_df)
#joined.to_csv("mercedes.csv", index=False)

In [44]:
#test model accuracy
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error


print("mean squared error:" ,mean_squared_error(output, y_test))
print("R2 score:" ,r2_score(output,y_test))

mean squared error: 0.17285397295983146
R2 score: -0.5217310213164306


In [120]:
#load file data into dictionnary 
with open('Microblog_Trialdata.json') as json_data:
    print(type(json_data))
    data_dict = json.load(json_data)

<class '_io.TextIOWrapper'>


In [121]:
len(data_dict)

16